**AI 노바투스과정**

Week 3: 딥러닝 기초 I (2023-01-27)

실습 #4: 깊은 신경망 학습에 필요한 기술 익히기

In [ ]:
# keras 라이브러리 가져오기
# df
# import keras -> import tensorflow.keras as keras
import tensorflow.keras as keras

In [ ]:
# MNIST 패션 데이터 로딩하기

fashion_mnist = keras.datasets.fashion_mnist

In [ ]:
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [ ]:
# 라벨의 각 숫자값에 해당하는 클래스 명 지정하기

class_names=["T-shirt/top",'Trouser','Pullover','Dress',"Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [ ]:
# 학습용 데이터를 다시 학습용 데이터와 검증용 데이터 (valdiation set) 으로 분리
# 분리와 동시에 정규화

X_valid, X_train= X_train_full[:5000]/255.0, X_train_full[5000:]/255.0  # 5000개 까지 validation data, 55000개는 train data
y_valid, y_train= y_train_full[:5000], y_train_full[5000:]

X_test=X_test/255.0

## 1. 다양한 활성함수 사용해보기

In [ ]:
# keras 의 Sequential API 를 활용하여 신경망 모형의 골격 만들기 
# 은닉층이 2개이며 첫번째 은닉층에 뉴런 300개, 두번째 은닉층에 뉴런 100개 인 골격 만들기
### 새로운 활성함수 사용해보기 !! 

model = keras.models.Sequential()  # 모델 초기화

model.add(keras.layers.Flatten(input_shape=[28,28])) # 28x28 2차원 이미지의 데이터를 1차원데이터로
model.add(keras.layers.Dense(300, activation="selu"))
model.add(keras.layers.Dense(100, activation="elu"))
model.add(keras.layers.Dense(10,  activation="softmax")) #10개의 class에 대한 확률값

# https://keras.io/ko/activations/ 

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_3 (Dense)             (None, 300)               235500    
                                                                 
 dense_4 (Dense)             (None, 100)               30100     
                                                                 
 dense_5 (Dense)             (None, 10)                1010      
                                                                 
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


## 2. 다양한 최적화 방법 사용해보기

In [ ]:
# 모형 컴파일하기 (손실함수, 최적화 방법 지정하기)
### 다양한 최적화 방법 사용해보기 !

# https://keras.io/ko/optimizers/ 

#optim = keras.optimizers.SGD(clipvalue=1.0)
#optim = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)  # momentum
#optim = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9, nesterov=True)  # NAG
#optim = keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9)  # RMSprop
#optim = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)  # Adam
#model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

optim = keras.optimizers.Adam(learning_rate=0.001)  # 학습률 지정 가능 
model.compile(loss="sparse_categorical_crossentropy", optimizer=optim, metrics=["accuracy"])


In [ ]:
#모형 학습하기

history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 8s 4ms/step - loss: 0.4835 - accuracy: 0.8245 - val_loss: 0.3784 - val_accuracy: 0.8616
Epoch 2/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.3777 - accuracy: 0.8614 - val_loss: 0.4014 - val_accuracy: 0.8604
Epoch 3/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.3428 - accuracy: 0.8717 - val_loss: 0.4139 - val_accuracy: 0.8464
Epoch 4/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.3205 - accuracy: 0.8797 - val_loss: 0.3761 - val_accuracy: 0.8682
Epoch 5/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.3007 - accuracy: 0.8872 - val_loss: 0.3494 - val_accuracy: 0.8808
Epoch 6/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.2848 - accuracy: 0.8936 - val_loss: 0.3231 - val_accuracy: 0.8872
Epoch 7/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.2774 - accuracy: 0.8939 - val_loss: 0.3177 - val_accuracy:

## 3. Dropout 사용해보기

In [ ]:
# Dropout : 의도적으로 노드와 노드 사이의 연결을 랜덤으로 끊고 학습시키는 방법. 
# 노드와 노드 사이의 연결을 끊어 특정 가중치에 과하게 학습되지 않도록 함. --> 과적합 방지 

model = keras.models.Sequential()

model.add(keras.layers.Flatten(input_shape=[28,28]))
model.add(keras.layers.Dropout(rate=0.2))   # 20%의 확률로 레이어 노드간 연결을 끊음 
model.add(keras.layers.Dense(300,activation="elu"))
model.add(keras.layers.Dropout(rate=0.2))
model.add(keras.layers.Dense(100,activation="elu"))
model.add(keras.layers.Dropout(rate=0.2))
model.add(keras.layers.Dense(10,activation="softmax"))

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])


In [ ]:
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 8s 4ms/step - loss: 0.7670 - accuracy: 0.7282 - val_loss: 0.5129 - val_accuracy: 0.8230
Epoch 2/10
1719/1719 [==============================] - 9s 5ms/step - loss: 0.5701 - accuracy: 0.7962 - val_loss: 0.4642 - val_accuracy: 0.8378
Epoch 3/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.5317 - accuracy: 0.8081 - val_loss: 0.4440 - val_accuracy: 0.8476
Epoch 4/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.5070 - accuracy: 0.8169 - val_loss: 0.4242 - val_accuracy: 0.8526
Epoch 5/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.4942 - accuracy: 0.8211 - val_loss: 0.4126 - val_accuracy: 0.8556
Epoch 6/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.4829 - accuracy: 0.8239 - val_loss: 0.4029 - val_accuracy: 0.8600
Epoch 7/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.4741 - accuracy: 0.8281 - val_loss: 0.4026 - val_accuracy:

## 4. Batch Normalization 사용해보기

In [ ]:
# 배치 정규화 : 배치 단위로 신경망을 학습시키는데, 각 배치 입력 데이터들을 정규화하여 Gradient Descent 학습의 안정성 향상 
# 학습 과정에서 Batch마다 평균과 분산을 활용하여 데이터의 분포를 정규화하는 과정. 데이터 종류에 따라 값의 범위는 천차만별이기 때문에, Batch마다 입력값의 범위를 스케일링. 

model=keras.models.Sequential()

model.add(keras.layers.Flatten(input_shape=[28,28]))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(300,activation="elu"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(100,activation="elu"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(10,activation="softmax"))

In [ ]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_4 (Flatten)         (None, 784)               0         
                                                                 
 batch_normalization (BatchN  (None, 784)              3136      
 ormalization)                                                   
                                                                 
 dense_12 (Dense)            (None, 300)               235500    
                                                                 
 batch_normalization_1 (Batc  (None, 300)              1200      
 hNormalization)                                                 
                                                                 
 dense_13 (Dense)            (None, 100)               30100     
                                                                 
 batch_normalization_2 (Batc  (None, 100)             

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])


In [ ]:
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 11s 6ms/step - loss: 0.5384 - accuracy: 0.8123 - val_loss: 0.4057 - val_accuracy: 0.8590
Epoch 2/10
1719/1719 [==============================] - 10s 6ms/step - loss: 0.4186 - accuracy: 0.8517 - val_loss: 0.3763 - val_accuracy: 0.8662
Epoch 3/10
1719/1719 [==============================] - 10s 6ms/step - loss: 0.3804 - accuracy: 0.8639 - val_loss: 0.3544 - val_accuracy: 0.8780
Epoch 4/10
1719/1719 [==============================] - 10s 6ms/step - loss: 0.3613 - accuracy: 0.8703 - val_loss: 0.3451 - val_accuracy: 0.8810
Epoch 5/10
1719/1719 [==============================] - 10s 6ms/step - loss: 0.3403 - accuracy: 0.8778 - val_loss: 0.3400 - val_accuracy: 0.8794
Epoch 6/10
1719/1719 [==============================] - 11s 6ms/step - loss: 0.3246 - accuracy: 0.8826 - val_loss: 0.3261 - val_accuracy: 0.8834
Epoch 7/10
1719/1719 [==============================] - 10s 6ms/step - loss: 0.3106 - accuracy: 0.8874 - val_loss: 0.3216 - val_ac

## 5. L1, L2 규제 사용해보기

In [ ]:
# 가중치 규제 (Weight Regularization) : 
# 최소화 하려는 손실함수에 가중치들의 합을 더해주어, 가중치들의 합도 같이 최소화. 특정 가중치에 과하게 학습되는것을 방지 --> 과적합 방지 
# L1 규제 : 가중치들의 절댓값 합 적용 
# L2 규제 : 가중치들의 제곱 합 적용 : 규제 강도가 더 강함

model = keras.models.Sequential()

model.add(keras.layers.Flatten(input_shape=[28,28]))
model.add(keras.layers.Dense(300,activation="elu"))
model.add(keras.layers.Dense(100,activation="elu", kernel_regularizer=keras.regularizers.l1(0.01)))
# model.add(keras.layers.Dense(100,activation="elu", kernel_regularizer=keras.regularizers.l2(0.01)))
model.add(keras.layers.Dense(10,activation="softmax"))



In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])


In [ ]:
history=model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 7s 4ms/step - loss: 5.6396 - accuracy: 0.7399 - val_loss: 1.2175 - val_accuracy: 0.7520
Epoch 2/10
1719/1719 [==============================] - 7s 4ms/step - loss: 1.0294 - accuracy: 0.7576 - val_loss: 0.8759 - val_accuracy: 0.7828
Epoch 3/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.8435 - accuracy: 0.7783 - val_loss: 0.8070 - val_accuracy: 0.7880
Epoch 4/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.7584 - accuracy: 0.7934 - val_loss: 0.7126 - val_accuracy: 0.8068
Epoch 5/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.7049 - accuracy: 0.8055 - val_loss: 0.6742 - val_accuracy: 0.8186
Epoch 6/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.6695 - accuracy: 0.8127 - val_loss: 0.6397 - val_accuracy: 0.8268
Epoch 7/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.6436 - accuracy: 0.8175 - val_loss: 0.6582 - val_accuracy: